In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="show the raw code."></form>''')

# NRS - real time nodes prediction
This experiment demonstrates the applicability of the **TSAF** framework evaluating forecasting models to the problem of
forecasting oceanographic sensor's network data streams as a component of an automatic quality data control.

The experiment uses the data from the real-time nodes from the Australia's National Reference Stations (NRS), a large
scale, long-term baseline about the Australian coastal seas.

There are four real-time nodes in operation on the NRS infrastructure, where each one provides data
at least since the last decade. Those nodes cover water quality parameters (condutivity, temperature, depth, fluorescence,
turbidity, oxygen) and water column current velocity, direction, and directional wave parameters. The following map
describes the location of each node and the maximum nominal depth of the sensors in the array.

## Data
The data was source from the IMOS infrastructure, through the use of the
[IMOS Aggregated Time Series Product](https://github.com/aodn/python-aodntools/blob/1.4.1/aodntools/timeseries_products/Documentation/aggregated_timeseries.md),
a aggregated file for each station and parameter, using the integral data except out-of-water data.

For each file, we applied a pre-processing step that generate a time-series for each variable of interest in each nominal
depth. The processing applied is available on a public github [repository](https://github.com/santinoalves/TSAF/blob/main/example/realTimeNRSNodes.py).

For each of the sites, the TSAF have processed the variables chlorophyll concentration, dissolved oxygen, salinity, and
turbidity that would be the totality of the non-direction data available.

The first step of the analysis, based on the TSAF framework is the identification of the data characteristics. The
following table describes the characteristics of the data based on the function identify_characteristics from the
[problemDefinition](https://github.com/santinoalves/TSAF/blob/main/TSAF/problemDefinition.py) module of the TSAF framework

In [2]:
import pandas as pd
import numpy as np
from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh import tile_providers as tiles
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *
import os
from TSAF import problemDefinition as problem
from ipywidgets import interact, widgets
from IPython.display import display, HTML

%matplotlib widget
import matplotlib.pyplot as plt

frame_summary = pd.DataFrame()

def display_summary(out, frame_summary,convert_to_date=False):
    out.clear_output()
    day_in_seconds = 60*60*24
    if convert_to_date:
        frame_summary[['beginning','end']] = frame_summary[['beginning','end']].apply(lambda x: pd.to_datetime(arg=x,unit='ms'))
    frame_summary['%missing'] = frame_summary.apply(lambda x: 100*x[4]/x[0], axis = 1)
    frame_summary['%cont'] = frame_summary.apply(lambda x: 100*x[5]/x[0], axis = 1)
    frame_summary['days_cont'] = frame_summary.apply(lambda x:(x[5]*x[3])/day_in_seconds, axis = 1 )
    out.append_display_data(HTML(frame_summary.drop(['period','continuos_observations'], axis=1).rename(columns={'null_observations': '#missing','continuos_observations':'continuous'}).sort_values(by=['variable','depth']).to_html()))

def summary(object):
    
    out.clear_output()
    with out:
        print("Calculating the summary of the time series from station: "+data+" \n it may take a while ")
    
    
    dir = current_path = os.getcwd()
    
    summary_path = os.path.join(dir, 'summary', )
    summary_file_name_with_path=os.path.join(summary_path,'sumary_'+data+'.json')
    
    global frame_summary
    if os.path.isfile(summary_file_name_with_path):
        frame_summary = pd.read_json(path_or_buf=summary_file_name_with_path)
        if len(frame_summary) > 0 :
            display_summary(out,frame_summary,True)
            return
    else:
        frame_summary = pd.DataFrame()
        
    data_path = os.path.join(dir, 'data', )
    files_list = os.listdir(data_path)
    files_list2 = filter(lambda x: data in x , files_list)
    index = 0
    list = []
    error = 0
    for x in files_list2:
        try:
            
            file_name_with_path = os.path.join(data_path,x)
            timeSeries = problem.load_json_time_series(file_name_with_path)
            parts = x.split("@")
            summary = problem.summary_series(timeSeries)
            summary["site"] = parts[0]
            summary["variable"] = parts[1]
            summary["depth"] = parts[5].replace(".json","")
            list.append(summary)
        except:
            error = error+1
    
    frame_summary = pd.DataFrame(list)
    frame_summary.to_json(summary_file_name_with_path)
    if len(frame_summary) > 0 :
            display_summary(out,frame_summary,False)
    with out:
        print("quantity of errors: "+str(error))

def showControles():
    button = widgets.Button(
    description='calculate summary',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='calculate summary',
    icon='check' # (FontAwesome names without the `fa-` prefix)
    )
    button.on_click(summary)
    display(button)
    return button
    
    
def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

def hex_map(plot,df, scale,leg_label='Hexbin Heatmap'):
    source=ColumnDataSource(df)
    station_render_objects=plot.circle(fill_alpha = 0.5,source=source,size=scale,hover_color='pink',hover_alpha=0.8,legend_label=leg_label)
    hex_hover = HoverTool(tooltips=[('Station','@Station'),('Lat:','@Lat'),('Long:','@Long'),('Nominal Depth:','@depth')],mode='mouse',point_policy='follow_mouse',renderers=[station_render_objects])
    hex_hover.renderers.append(station_render_objects)
    plot.tools.append(hex_hover)

    plot.legend.location = "top_right"
    plot.legend.click_policy="hide"
    callback = CustomJS(args=dict(station_render_objects=station_render_objects, source=source),code="""
        console.log('Tap event occurred at object: ' + station_render_objects.data_source.selected)
        console.log('Tap event occurred at x-position: ' + cb_obj.x)
        console.log('Tap event occurred at value: ' + cb_obj.value)
        var var_name = 'data';
        var minDist = Number.MAX_VALUE
        var position = -1
        for (var i = 0; i < station_render_objects.data_source.data.x.length; i++) {
            var obj_x = station_render_objects.data_source.data.x[i];
            var obj_y = station_render_objects.data_source.data.y[i];
            var dist = Math.pow(Math.pow(obj_x - cb_obj.x,2) + Math.pow(obj_y - cb_obj.y,2),0.5);
            console.log("object: "+i+" dist: "+dist);
            if (dist <= minDist) {
                minDist = dist;
                position = i
            }
            
        }
        console.log ("position: "+position)
        var value = station_render_objects.data_source.data.Reference[position];
        var command = var_name + " = " + "'"+value+"'";
        var command2 = "button.disabled = False";
        console.log("Executing Command: " + command);

        var kernel = IPython.notebook.kernel;
        kernel.execute(command);
        console.log("executing Command: "+command2);
        kernel.execute(command2);
        """)

    plot.js_on_event('tap', callback)

source_file = 'sources.csv'
df = pd.read_csv(source_file, delimiter=",",index_col=0)
source_mercator=wgs84_to_web_mercator(df,'Long','Lat')
k = 6378137

y_min =  np.log(np.tan((90 + -45.26581) * np.pi/360.0)) * k
y_max = np.log(np.tan((90 + -9.34962) * np.pi/360.0)) * k
x_min = 108.17368 * (k * np.pi/180.0)
x_max = 156.03012 * (k * np.pi/180.0)
tile_provider=tiles.get_provider('OSM')
plot=figure(
    title='Australia\'s National Reference Stations - Online Moored Sensor arrays',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save,tap',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator'
    )
plot.grid.visible=True

map=plot.add_tile(tile_provider)
map.level='underlay'

plot.xaxis.visible = False
plot.yaxis.visible=False

hex_map(plot=plot,
        df=source_mercator,
        scale=10,
        leg_label='Moorings position')




output_notebook()
show(plot)
data = "No information selected, please, select a station and try again."
button = showControles()
out = widgets.Output(layout={'border': '1px solid black'})
with out:
    print("Select a mooring to start the analysis")
display(out)


ModuleNotFoundError: No module named 'TSAF'

Additionally, for each time series, the components' analysis will describe de properties related to stationarity, cycle
and trend. To perform the analysis, the TSAF framework offers the function

In [7]:
from TSAF.base import Series as Series


def showanalysis(site,variable,depth):
    selected = tab_nest.selected_index
    tab_nest.set_title(selected, "Analysis "+str(selected+1)+" - "+controls[selected][0].value)
    out = controls[selected][-1]
    out.clear_output()
    with out:
        print("calculating features of "+controls[selected][0].value+" ...")
        for file_name in files_list2:
            if file_name.find(site) > -1 and file_name.find(variable) > -1 and file_name.find(depth+".json") > -1:
                file_name_with_path = os.path.join(data_path,file_name)
                print("   importing time series ...")
                timeSeries = problem.load_json_time_series(file_name_with_path)
                
                trimmed_time_series = Series.Series(timeSeries).trim_time_series
                characteristics = problem.identify_time_series_characteristics(trimmed_time_series)
                charac = pd.DataFrame.from_dict(characteristics, orient='index')
                components_cycle = problem.analysis_components(time_series = timeSeries, show_analysis = False)
                cycle_ = pd.DataFrame.from_dict(components_cycle, orient='index')
                cycle_['Error (Seasonal component)'] = cycle_.apply(lambda x:np.mean(x['scores']), axis = 1 )
                
                out.clear_output()
                
                
                a= widgets.Output()
                a.append_display_data(HTML(cycle_['Error (Seasonal component)'].to_frame().to_html()))
                b= widgets.Output()
                b.append_display_data(HTML(charac.to_html()))
                display(widgets.HBox([b,a]))
                
                
                fig, ax = plt.subplots()
                fig.canvas.capture_scroll = True
                ax.plot(trimmed_time_series.index, trimmed_time_series.values)
                ax.set(xlabel='date/time', ylabel=controls[selected][0].value,
                       title='Original Time Series')
                ax.grid()
                fig.canvas.toolbar_visible = True
                #display(fig)

                type_func = None
                if cycle_['Error (Seasonal component)']['poly'] < cycle_['Error (Seasonal component)']['linear']:
                    type_func = 'poly'
                else:
                    type_func = 'linear'
                    
                time_series_deviation = problem.remove_seasonality(timeSeries,type_func)
                
                characteristics = problem.identify_time_series_characteristics(time_series_deviation)
                charac = pd.DataFrame.from_dict(characteristics, orient='index')
                components_cycle = problem.analysis_components(time_series = time_series_deviation, show_analysis = False)
                cycle_ = pd.DataFrame.from_dict(components_cycle, orient='index')
                cycle_['Error (Seasonal component)'] = cycle_.apply(lambda x:np.mean(x['scores']), axis = 1 )
                
                a= widgets.Output()
                a.append_display_data(HTML(cycle_['Error (Seasonal component)'].to_frame().to_html()))
                b= widgets.Output()
                b.append_display_data(HTML(charac.to_html()))
                display(widgets.HBox([b,a]))
                
                
                fig1, ax = plt.subplots()
                fig1.canvas.capture_scroll = True
                ax.plot(time_series_deviation.index, time_series_deviation.values)
                ax.set(xlabel='date/time', ylabel=controls[selected][0].value,
                       title='Time Series minus seasonal effect')
                ax.grid()
                fig.canvas.toolbar_visible = True
                #plt.show()
                #display(fig)
                #describe components
                
                if characteristics['stationary'] != 'Population':
                    
                    time_series_deviation = problem.remove_seasonality(timeSeries,'diff')
                
                    characteristics = problem.identify_time_series_characteristics(time_series_deviation)
                    charac = pd.DataFrame.from_dict(characteristics, orient='index')
                    components_cycle = problem.analysis_components(time_series = time_series_deviation, show_analysis = False)
                    cycle_ = pd.DataFrame.from_dict(components_cycle, orient='index')
                    cycle_['Error (Seasonal component)'] = cycle_.apply(lambda x:np.mean(x['scores']), axis = 1 )

                    a= widgets.Output()
                    a.append_display_data(HTML(cycle_['Error (Seasonal component)'].to_frame().to_html()))
                    b= widgets.Output()
                    b.append_display_data(HTML(charac.to_html()))
                    display(widgets.HBox([b,a]))


                    fig2, ax = plt.subplots()
                    fig2.canvas.capture_scroll = True
                    ax.plot(time_series_deviation.index, time_series_deviation.values)
                    ax.set(xlabel='date/time', ylabel=controls[selected][0].value,
                           title='differenced Time Series')
                    ax.grid()
                    fig2.canvas.toolbar_visible = True
                    #plt.show()
                    #display(fig)
                
                
                
               # out.append_display_data()
                #details stationarity tests and conclusion
                #process TS if needed
                #plot processed TS
                #details stationarity tests and conclusions
                #show table of properties of the time series
                return 

def perform_analysis(object):
    selected = tab_nest.selected_index
    dropdown = controls[selected][0]
    selection = dropdown.value
    [variable,depth] = selection.split(' ')
    showanalysis(data,variable,depth)
    
dir = current_path = os.getcwd()
data_path = os.path.join(dir, 'data', )
files_list = os.listdir(data_path)
files_list2 = filter(lambda x: data in x , files_list)
summary_index = pd.DataFrame()
summary_index['label'] = frame_summary.sort_values(by=['variable','depth'])[['variable','depth']].apply((lambda x: x[0]+' '+str(x[1])), axis=1)


def create_visual_components():
    dropdown = widgets.Dropdown(
        options=summary_index['label'],
        description='Variable:',
        disabled=False,
    )
    button = widgets.Button(
        description='perform analysis',
        disabled=False,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='perform analysis',
        icon='check' # (FontAwesome names without the `fa-` prefix)
        )
    button.on_click(perform_analysis)
    out = widgets.Output(layout={'border': '1px solid black'})
    visual_input = widgets.Output(layout={'border': '1px solid black'})
    visual_input=widgets.HBox([dropdown, button])
    accordion = widgets.VBox([visual_input, out])
    return [dropdown, accordion, out]


controls=[]
number_of_tabs = 6
for i in range(number_of_tabs):
    controls.insert(i,create_visual_components())

tab_nest = widgets.Tab()
tab_nest.children =[controls[i][1] for i in range(number_of_tabs)]
[tab_nest.set_title(i, "Analysis "+str(i+1)) for i in range(number_of_tabs)]
display(tab_nest)